In [1]:
from numpy.random import seed
seed(7532)
from tensorflow import set_random_seed
set_random_seed(7532)

import numpy as np
import pandas as pd

from keras import Input
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.layers import (BatchNormalization, Dense, Dropout, Concatenate, Conv2D, Flatten, 
                          MaxPool2D)

Using TensorFlow backend.


Read in the data.

In [2]:
train_set = pd.read_csv('train_set_metadata.csv')
valid_set = pd.read_csv('valid_set_metadata.csv')
test_set = pd.read_csv('test_set_metadata.csv')

train_set_partition = np.load('train_set_partition.npy')

Compute the number of parts the train set was partitioned into.

In [3]:
n_parts = len(train_set_partition) - 1
n_parts

20

## CNN Model Training from Scratch (No Data Augmentation)##

In [4]:
BEST_MODEL_PATH = 'best_model.hdf5'
INPUT_SHAPE = (320, 320, 3)

LEARNING_RATE = 0.0001
N_EPOCHS = 50
BATCH_SIZE = 32

In [5]:
def create_nonsequential_model(input_shape):
    input_data = Input(shape=input_shape)

    
    left_leg   = Conv2D(16, kernel_size=3, activation='relu', padding='same')(input_data)
    left_leg   = BatchNormalization()(left_leg)    
    left_leg   = Conv2D(16, kernel_size=3, activation='relu', padding='same')(left_leg)    

    right_leg  = Conv2D(16, kernel_size=5, activation='relu', padding='same')(input_data)
    right_leg  = BatchNormalization()(right_leg)
    right_leg  = Conv2D(16, kernel_size=5, activation='relu', padding='same')(right_leg)
    
    concat     = Concatenate()([left_leg, right_leg])
    max_pool   = MaxPool2D(2)(concat)
    batch_norm = BatchNormalization()(max_pool)
    
    
    left_leg   = Conv2D(32, kernel_size=3, activation='relu', padding='same')(batch_norm)
    left_leg   = BatchNormalization()(left_leg)    
    left_leg   = Conv2D(32, kernel_size=3, activation='relu', padding='same')(left_leg)    

    right_leg  = Conv2D(32, kernel_size=5, activation='relu', padding='same')(batch_norm)
    right_leg  = BatchNormalization()(right_leg)
    right_leg  = Conv2D(32, kernel_size=5, activation='relu', padding='same')(right_leg)
    
    concat     = Concatenate()([left_leg, right_leg])
    max_pool   = MaxPool2D(2)(concat)
    batch_norm = BatchNormalization()(max_pool)


    left_leg   = Conv2D(64, kernel_size=3, activation='relu', padding='same')(batch_norm)
    left_leg   = BatchNormalization()(left_leg)    
    left_leg   = Conv2D(64, kernel_size=3, activation='relu', padding='same')(left_leg)    

    right_leg  = Conv2D(64, kernel_size=5, activation='relu', padding='same')(batch_norm)
    right_leg  = BatchNormalization()(right_leg)
    right_leg  = Conv2D(64, kernel_size=5, activation='relu', padding='same')(right_leg)
    
    concat     = Concatenate()([left_leg, right_leg])
    max_pool   = MaxPool2D(2)(concat)
    batch_norm = BatchNormalization()(max_pool)
    

    left_leg   = Conv2D(128, kernel_size=3, activation='relu', padding='same')(batch_norm)
    left_leg   = BatchNormalization()(left_leg)    
    left_leg   = Conv2D(128, kernel_size=3, activation='relu', padding='same')(left_leg)    

    right_leg  = Conv2D(128, kernel_size=5, activation='relu', padding='same')(batch_norm)
    right_leg  = BatchNormalization()(right_leg)
    right_leg  = Conv2D(128, kernel_size=5, activation='relu', padding='same')(right_leg)
    
    concat     = Concatenate()([left_leg, right_leg])
    max_pool   = MaxPool2D(5)(concat)
    batch_norm = BatchNormalization()(max_pool)    
       
    
    flatten   = Flatten()(batch_norm)
    dropout   = Dropout(0.5)(flatten)
    dense     = Dense(512, activation='relu', kernel_regularizer=l2(0.000001))(dropout)
    dropout   = Dropout(0.5)(dense)
    dense     = Dense(512, activation='relu', kernel_regularizer=l2(0.000001))(dropout)
    dropout   = Dropout(0.5)(dense)
    dense     = Dense(1, activation='sigmoid')(dropout)
    
    
    model = Model(inputs=input_data, outputs=dense)   
    
    return model

In [6]:
model = create_nonsequential_model(INPUT_SHAPE)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 320, 320, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 320, 320, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 320, 320, 16) 1216        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 320, 320, 16) 64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_norm

In [7]:
gender_column_position = train_set.columns.get_loc('gender')

X_valid = np.load('valid_set_hmgd_arr.npy')
y_valid = valid_set['gender'].values

for part in range(n_parts):        
    print(f'Part {part + 1}:')
    
    train_filename = 'train_set_hmgd_arr_' + str(part + 1).zfill(2) + '.npy'
    subrange = range(train_set_partition[part], train_set_partition[part + 1])    
    X_train = np.load(train_filename)
    y_train = train_set.iloc[subrange, gender_column_position].values
    
    # Model initialization/load
    if part:
        model = load_model(BEST_MODEL_PATH)
    else:
        model = create_nonsequential_model(X_train.shape[1:])
        adam = Adam(lr=LEARNING_RATE)
        model.compile(optimizer=adam, loss='binary_crossentropy', 
                      metrics=['binary_accuracy'])
        
        # Initialize callbacks
        checkpoint = ModelCheckpoint(BEST_MODEL_PATH, 
                                     monitor='val_loss', 
                                     save_best_only=True, 
                                     save_weights_only=False)
        
        lr_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                         factor=0.2, 
                                         patience=6)
        
        early_stopping = EarlyStopping(monitor='val_loss', 
                                       patience=12)
        
        callback_list = [checkpoint, lr_reduction, early_stopping]
    
    model.fit(X_train, y_train, epochs=N_EPOCHS, batch_size=BATCH_SIZE, 
              callbacks=callback_list, validation_data=(X_valid, y_valid))
    
    #free up memory
    del X_train

Part 1:
Train on 8271 samples, validate on 4176 samples
Epoch 1/50
8271/8271 [==============================] - 323s 39ms/step - loss: 0.8421 - binary_accuracy: 0.6599 - val_loss: 0.5939 - val_binary_accuracy: 0.7414
Epoch 2/50
8271/8271 [==============================] - 287s 35ms/step - loss: 0.6561 - binary_accuracy: 0.7159 - val_loss: 0.5379 - val_binary_accuracy: 0.7636
Epoch 3/50
8271/8271 [==============================] - 286s 35ms/step - loss: 0.5526 - binary_accuracy: 0.7584 - val_loss: 0.4739 - val_binary_accuracy: 0.7809
Epoch 4/50
8271/8271 [==============================] - 287s 35ms/step - loss: 0.4906 - binary_accuracy: 0.7784 - val_loss: 0.4881 - val_binary_accuracy: 0.7804
Epoch 5/50
8271/8271 [==============================] - 286s 35ms/step - loss: 0.4341 - binary_accuracy: 0.8111 - val_loss: 0.4919 - val_binary_accuracy: 0.7608
Epoch 6/50
8271/8271 [==============================] - 287s 35ms/step - loss: 0.4069 - binary_accuracy: 0.8217 - val_loss: 0.4377 - val_bi

Epoch 11/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.1166 - binary_accuracy: 0.9570 - val_loss: 0.3929 - val_binary_accuracy: 0.8506
Epoch 12/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.1029 - binary_accuracy: 0.9632 - val_loss: 0.4001 - val_binary_accuracy: 0.8487
Epoch 13/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.0973 - binary_accuracy: 0.9642 - val_loss: 0.4210 - val_binary_accuracy: 0.8479
Epoch 14/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.0804 - binary_accuracy: 0.9710 - val_loss: 0.4503 - val_binary_accuracy: 0.8386
Epoch 15/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.0799 - binary_accuracy: 0.9703 - val_loss: 0.4426 - val_binary_accuracy: 0.8520
Part 4:
Train on 8271 samples, validate on 4176 samples
Epoch 1/50
8271/8271 [==============================] - 296s 36ms/step - loss: 0.3540 - binary_accuracy: 0.8533 - val_loss: 0.3569 - v

Epoch 14/50
8271/8271 [==============================] - 289s 35ms/step - loss: 0.0561 - binary_accuracy: 0.9813 - val_loss: 0.4431 - val_binary_accuracy: 0.8534
Epoch 15/50
8271/8271 [==============================] - 289s 35ms/step - loss: 0.0520 - binary_accuracy: 0.9823 - val_loss: 0.4621 - val_binary_accuracy: 0.8518
Epoch 16/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.0506 - binary_accuracy: 0.9837 - val_loss: 0.4781 - val_binary_accuracy: 0.8542
Epoch 17/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.0434 - binary_accuracy: 0.9867 - val_loss: 0.4813 - val_binary_accuracy: 0.8551
Part 7:
Train on 8271 samples, validate on 4176 samples
Epoch 1/50
8271/8271 [==============================] - 296s 36ms/step - loss: 0.3236 - binary_accuracy: 0.8662 - val_loss: 0.3497 - val_binary_accuracy: 0.8558
Epoch 2/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.2853 - binary_accuracy: 0.8851 - val_loss: 0.3384 - va

Epoch 16/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.0342 - binary_accuracy: 0.9896 - val_loss: 0.5130 - val_binary_accuracy: 0.8546
Epoch 17/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.0335 - binary_accuracy: 0.9900 - val_loss: 0.5256 - val_binary_accuracy: 0.8590
Part 10:
Train on 8271 samples, validate on 4176 samples
Epoch 1/50
8271/8271 [==============================] - 298s 36ms/step - loss: 0.3255 - binary_accuracy: 0.8687 - val_loss: 0.3681 - val_binary_accuracy: 0.8678
Epoch 2/50
8271/8271 [==============================] - 289s 35ms/step - loss: 0.2745 - binary_accuracy: 0.8893 - val_loss: 0.3309 - val_binary_accuracy: 0.8661
Epoch 3/50
8271/8271 [==============================] - 289s 35ms/step - loss: 0.2596 - binary_accuracy: 0.8946 - val_loss: 0.3292 - val_binary_accuracy: 0.8671
Epoch 4/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.2278 - binary_accuracy: 0.9115 - val_loss: 0.3271 - val

Part 13:
Train on 8271 samples, validate on 4176 samples
Epoch 1/50
8271/8271 [==============================] - 301s 36ms/step - loss: 0.3106 - binary_accuracy: 0.8712 - val_loss: 0.3668 - val_binary_accuracy: 0.8594
Epoch 2/50
8271/8271 [==============================] - 290s 35ms/step - loss: 0.2665 - binary_accuracy: 0.8942 - val_loss: 0.3231 - val_binary_accuracy: 0.8657
Epoch 3/50
8271/8271 [==============================] - 289s 35ms/step - loss: 0.2384 - binary_accuracy: 0.9045 - val_loss: 0.3209 - val_binary_accuracy: 0.8702
Epoch 4/50
8271/8271 [==============================] - 289s 35ms/step - loss: 0.2123 - binary_accuracy: 0.9133 - val_loss: 0.3238 - val_binary_accuracy: 0.8683
Epoch 5/50
8271/8271 [==============================] - 289s 35ms/step - loss: 0.1879 - binary_accuracy: 0.9235 - val_loss: 0.3381 - val_binary_accuracy: 0.8580
Epoch 6/50
8271/8271 [==============================] - 289s 35ms/step - loss: 0.1623 - binary_accuracy: 0.9386 - val_loss: 0.3624 - val_b

Epoch 6/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.1708 - binary_accuracy: 0.9353 - val_loss: 0.3332 - val_binary_accuracy: 0.8645
Epoch 7/50
8271/8271 [==============================] - 287s 35ms/step - loss: 0.1403 - binary_accuracy: 0.9452 - val_loss: 0.3419 - val_binary_accuracy: 0.8664
Epoch 8/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.1238 - binary_accuracy: 0.9541 - val_loss: 0.3645 - val_binary_accuracy: 0.8666
Epoch 9/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.1006 - binary_accuracy: 0.9635 - val_loss: 0.3942 - val_binary_accuracy: 0.8597
Epoch 10/50
8271/8271 [==============================] - 289s 35ms/step - loss: 0.0929 - binary_accuracy: 0.9652 - val_loss: 0.4386 - val_binary_accuracy: 0.8410
Epoch 11/50
8271/8271 [==============================] - 288s 35ms/step - loss: 0.0669 - binary_accuracy: 0.9763 - val_loss: 0.4284 - val_binary_accuracy: 0.8659
Epoch 12/50
8271/8271 [=========

Read in the test data and evaluate the model.

In [8]:
X_test = np.load('test_set_hmgd_arr.npy')

In [9]:
model.evaluate(X_test, test_set['gender'].values)

7621/7621 [==============================] - 83s 11ms/step


[0.5070486121273639, 0.8547434719853038]

In [10]:
# free up memory
del X_test